In [1]:
from transformers import Trainer
from datasets import load_dataset
import torch
from datasets import DatasetDict, Dataset

from transformers import T5Tokenizer, T5ForConditionalGeneration

2024-12-11 23:30:36.611789: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733981437.079578    2783 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733981437.316131    2783 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 23:30:38.597770: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from transformers import TrainingArguments, Trainer

In [3]:
ds = load_dataset("gokaygokay/prompt-enhancer-dataset")

In [4]:

# AutoModelForCausalLM converts the embedding output of your model to a word. 
# If you use AutoModel, it doesn't convert to a word; only spits out the embedding.

tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small") 

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
def tokenize_function(examples):
    # Tokenize both the 'short_prompt' and 'long_prompt' columns
    short_prompt_tokens = tokenizer(examples["short_prompt"], padding="max_length", truncation=True)
    long_prompt_tokens = tokenizer(examples["long_prompt"], padding="max_length", truncation=True)
    
    # Merge both tokenized outputs into a single dictionary (or add them separately)
    return {
        'short_prompt_input_ids': short_prompt_tokens['input_ids'],
        'short_prompt_attention_mask': short_prompt_tokens['attention_mask'],
        'long_prompt_input_ids': long_prompt_tokens['input_ids'],
        'long_prompt_attention_mask': long_prompt_tokens['attention_mask']
    }

# Tokenize the entire dataset (train and test)
tokenized_datasets = ds.map(tokenize_function, batched=True)


In [6]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['short_prompt', 'long_prompt', 'short_prompt_input_ids', 'short_prompt_attention_mask', 'long_prompt_input_ids', 'long_prompt_attention_mask'],
        num_rows: 16110
    })
    test: Dataset({
        features: ['short_prompt', 'long_prompt', 'short_prompt_input_ids', 'short_prompt_attention_mask', 'long_prompt_input_ids', 'long_prompt_attention_mask'],
        num_rows: 1790
    })
})

In [7]:
def preprocess_data(examples):
    # The model takes input_ids and attention_mask as inputs, and long_prompt_input_ids as labels
    model_inputs = {
        'input_ids': examples['short_prompt_input_ids'],
        'attention_mask': examples['short_prompt_attention_mask'],
        'labels': examples['long_prompt_input_ids']  # The long_prompt_input_ids are the target labels
    }
    return model_inputs

# Apply the preprocessing to the train and test datasets
tokenized_datasets = tokenized_datasets.map(preprocess_data, batched=True)

In [8]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['short_prompt', 'long_prompt', 'short_prompt_input_ids', 'short_prompt_attention_mask', 'long_prompt_input_ids', 'long_prompt_attention_mask', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 16110
    })
    test: Dataset({
        features: ['short_prompt', 'long_prompt', 'short_prompt_input_ids', 'short_prompt_attention_mask', 'long_prompt_input_ids', 'long_prompt_attention_mask', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1790
    })
})

In [9]:

# Your dataset (replace with actual data)
dataset = DatasetDict({
    'train': Dataset.from_dict({
        'short_prompt_input_ids': tokenized_datasets['train']['short_prompt_input_ids'], 
        'short_prompt_attention_mask': tokenized_datasets['train']['short_prompt_attention_mask'], 
        'long_prompt_input_ids': tokenized_datasets['train']['long_prompt_input_ids']
    }),
    'test': Dataset.from_dict({
        'short_prompt_input_ids': tokenized_datasets['test']['short_prompt_input_ids'], 
        'short_prompt_attention_mask': tokenized_datasets['test']['short_prompt_attention_mask'], 
        'long_prompt_input_ids': tokenized_datasets['test']['long_prompt_input_ids']
    })
})


In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['short_prompt_input_ids', 'short_prompt_attention_mask', 'long_prompt_input_ids'],
        num_rows: 16110
    })
    test: Dataset({
        features: ['short_prompt_input_ids', 'short_prompt_attention_mask', 'long_prompt_input_ids'],
        num_rows: 1790
    })
})

In [11]:
training_args = TrainingArguments(
    output_dir="./t5_results",              # Save the model here
    evaluation_strategy="epoch",         # Evaluate after each epoch
    learning_rate=2e-5,                  # Learning rate
    per_device_train_batch_size=8,       # Training batch size
    per_device_eval_batch_size=8,        # Evaluation batch size
    num_train_epochs=1,                  # Number of epochs
    weight_decay=0.01,                   # Regularization
    logging_dir="./t5_logs",                # Logs directory
    logging_steps=10000,                   # Log every 2000 steps
)


/home/dsu/Project/PeacockFlock/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    tokenizer=tokenizer,
)


In [13]:
trainer.train()

  0%|          | 0/2014 [00:00<?, ?it/s]

  0%|          | 0/224 [00:00<?, ?it/s]

{'eval_loss': 0.8599253296852112, 'eval_runtime': 15.4866, 'eval_samples_per_second': 115.584, 'eval_steps_per_second': 14.464, 'epoch': 1.0}
{'train_runtime': 529.0872, 'train_samples_per_second': 30.449, 'train_steps_per_second': 3.807, 'train_loss': 1.2257741948935577, 'epoch': 1.0}


TrainOutput(global_step=2014, training_loss=1.2257741948935577, metrics={'train_runtime': 529.0872, 'train_samples_per_second': 30.449, 'train_steps_per_second': 3.807, 'total_flos': 2180356421713920.0, 'train_loss': 1.2257741948935577, 'epoch': 1.0})

In [14]:
trainer.evaluate() 

  0%|          | 0/224 [00:00<?, ?it/s]

{'eval_loss': 0.8599253296852112,
 'eval_runtime': 15.7372,
 'eval_samples_per_second': 113.743,
 'eval_steps_per_second': 14.234,
 'epoch': 1.0}

In [15]:
model.save_pretrained("./fine_tuned_t5")
tokenizer.save_pretrained("./fine_tuned_t5")


('./fine_tuned_t5/tokenizer_config.json',
 './fine_tuned_t5/special_tokens_map.json',
 './fine_tuned_t5/spiece.model',
 './fine_tuned_t5/added_tokens.json')

: 